# Interactive Spherical Coordinates Explorer

Use the sliders below to build intuition about how **elevation (θ)** and **azimuth (α)** angles map onto a hemisphere representing the sky dome.

**Key formulas:**
- `x = cos(θ) · cos(α)`
- `y = cos(θ) · sin(α)`  
- `z = sin(θ)` ← This determines brightness in the CIE sky model!


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, IntSlider
import ipywidgets as widgets

%matplotlib inline
plt.rcParams['figure.figsize'] = [14, 5]


## 1. Single Point Explorer

Move the sliders to see how a single point moves on the hemisphere as you change the angles.


In [ ]:
def spherical_to_cartesian(elevation_deg, azimuth_deg):
    """Convert spherical to Cartesian coordinates."""
    theta = np.radians(elevation_deg)
    alpha = np.radians(azimuth_deg)
    x = np.cos(theta) * np.cos(alpha)
    y = np.cos(theta) * np.sin(alpha)
    z = np.sin(theta)
    return x, y, z

def plot_single_point(elevation=45, azimuth=45):
    """Plot a single point on the hemisphere with given angles."""
    fig = plt.figure(figsize=(16, 6))

    # Get point coordinates
    x, y, z = spherical_to_cartesian(elevation, azimuth)

    # CIE luminance factor
    cie_factor = 1 + 2 * z

    # === 3D VIEW ===
    ax1 = fig.add_subplot(131, projection='3d')

    # Draw hemisphere wireframe
    u = np.linspace(0, 2*np.pi, 30)
    v = np.linspace(0, np.pi/2, 15)
    U, V = np.meshgrid(u, v)
    X_dome = np.cos(V) * np.cos(U)
    Y_dome = np.cos(V) * np.sin(U)
    Z_dome = np.sin(V)
    ax1.plot_wireframe(X_dome, Y_dome, Z_dome, alpha=0.2, color='gray')

    # Draw the point
    ax1.scatter([x], [y], [z], c='red', s=200, zorder=5, label='Your point')

    # Draw projection lines
    ax1.plot([x, x], [y, y], [0, z], 'r--', alpha=0.5, linewidth=2)
    ax1.plot([0, x], [0, y], [0, 0], 'b--', alpha=0.5, linewidth=2)
    ax1.plot([0, x], [0, y], [z, z], 'g--', alpha=0.5, linewidth=2)

    ax1.set_xlabel('X (East)')
    ax1.set_ylabel('Y (North)')
    ax1.set_zlabel('Z (Up)')
    ax1.set_title(f'3D View\nθ={elevation}°, α={azimuth}°')
    ax1.set_xlim(-1.2, 1.2)
    ax1.set_ylim(-1.2, 1.2)
    ax1.set_zlim(0, 1.2)

    # === TOP-DOWN VIEW ===
    ax2 = fig.add_subplot(132)

    # Horizon circle
    theta_circle = np.linspace(0, 2*np.pi, 100)
    ax2.plot(np.cos(theta_circle), np.sin(theta_circle), 'k-', alpha=0.3, linewidth=2)

    # Elevation rings
    for elev in [30, 60]:
        r = np.cos(np.radians(elev))
        ax2.plot(r*np.cos(theta_circle), r*np.sin(theta_circle), 'gray', linestyle='--', alpha=0.3)
        ax2.text(r+0.05, 0, f'θ={elev}°', fontsize=8, alpha=0.5)

    # The point (projected)
    ax2.scatter([x], [y], c='red', s=200, zorder=5)
    ax2.plot([0, x], [0, y], 'r-', linewidth=2, alpha=0.7)

    # Azimuth arc
    if azimuth > 0:
        arc_angles = np.linspace(0, np.radians(azimuth), 30)
        arc_r = 0.3
        ax2.plot(arc_r*np.cos(arc_angles), arc_r*np.sin(arc_angles), 'b-', linewidth=2)
        ax2.annotate(f'α={azimuth}°', xy=(arc_r*np.cos(np.radians(azimuth/2)),
                    arc_r*np.sin(np.radians(azimuth/2))), fontsize=12, color='blue')

    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_title(f'Top-Down View\nProjected radius r = cos({elevation}°) = {np.cos(np.radians(elevation)):.2f}')
    ax2.set_aspect('equal')
    ax2.set_xlim(-1.3, 1.3)
    ax2.set_ylim(-1.3, 1.3)
    ax2.axhline(y=0, color='gray', linewidth=0.5)
    ax2.axvline(x=0, color='gray', linewidth=0.5)

    # === SIDE VIEW ===
    ax3 = fig.add_subplot(133)

    # Dome profile
    profile_theta = np.linspace(0, np.pi/2, 50)
    ax3.plot(np.cos(profile_theta), np.sin(profile_theta), 'k-', linewidth=2, alpha=0.3)
    ax3.axhline(y=0, color='brown', linewidth=2, alpha=0.5)

    # Point (side view)
    r_proj = np.cos(np.radians(elevation))
    ax3.scatter([r_proj], [z], c='red', s=200, zorder=5)

    # Elevation angle arc
    elev_arc = np.linspace(0, np.radians(elevation), 30)
    arc_r = 0.4
    ax3.plot(arc_r*np.cos(elev_arc), arc_r*np.sin(elev_arc), 'g-', linewidth=2)
    ax3.annotate(f'θ={elevation}°', xy=(arc_r*np.cos(np.radians(elevation/2))+0.05,
                arc_r*np.sin(np.radians(elevation/2))), fontsize=12, color='green')

    # Height line
    ax3.plot([r_proj, r_proj], [0, z], 'r--', linewidth=2, alpha=0.7)
    ax3.annotate(f'z = sin({elevation}°) = {z:.2f}', xy=(r_proj+0.05, z/2), fontsize=11, color='red')

    ax3.set_xlabel('Horizontal Distance')
    ax3.set_ylabel('Height (z)')
    ax3.set_title(f'Side View\nCIE luminance factor = 1 + 2z = {cie_factor:.2f}')
    ax3.set_aspect('equal')
    ax3.set_xlim(-0.1, 1.3)
    ax3.set_ylim(-0.1, 1.2)

    # Add CIE info box
    textstr = f'CIE Sky Model:\nL ∝ (1 + 2z) = {cie_factor:.2f}\n\nBrightness relative\nto horizon: {cie_factor:.1f}x'
    ax3.text(0.02, 0.98, textstr, transform=ax3.transAxes, fontsize=10,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

    plt.tight_layout()
    plt.show()

    print(f"\n📍 Cartesian coordinates: x={x:.3f}, y={y:.3f}, z={z:.3f}")
    print(f"📐 Elevation θ = {elevation}° (how high in the sky)")
    print(f"🧭 Azimuth α = {azimuth}° (compass direction)")
    print(f"☀️ CIE luminance factor = 1 + 2×{z:.2f} = {cie_factor:.2f}")

# Create interactive widget
interact(
    plot_single_point,
    elevation=IntSlider(min=0, max=90, step=5, value=45, description='Elevation θ:'),
    azimuth=IntSlider(min=0, max=360, step=15, value=45, description='Azimuth α:')
);


interactive(children=(IntSlider(value=45, description='Elevation θ:', max=90, step=5), IntSlider(value=45, des…

---

## 2. Understanding Elevation (θ) - "How high in the sky?"

Slide the elevation to see how **z = sin(θ)** changes. This directly affects brightness in the CIE sky model!


In [7]:
def plot_elevation_exploration(elevation=45):
    """Show how elevation affects z-coordinate and CIE luminance."""
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))

    theta_rad = np.radians(elevation)
    z = np.sin(theta_rad)
    r = np.cos(theta_rad)
    cie = 1 + 2 * z

    # === Plot 1: Dome cross-section ===
    ax1 = axes[0]

    theta_range = np.linspace(0, np.pi/2, 100)
    ax1.plot(np.cos(theta_range), np.sin(theta_range), 'k-', linewidth=2)
    ax1.plot(-np.cos(theta_range), np.sin(theta_range), 'k-', linewidth=2)
    ax1.axhline(y=0, color='brown', linewidth=3)

    # Draw current elevation as a horizontal line at height z
    ax1.axhline(y=z, color='red', linewidth=2, linestyle='--', alpha=0.7)
    ax1.fill_between([-r, r], [z, z], [0, 0], alpha=0.2, color='red')
    ax1.scatter([r], [z], c='red', s=200, zorder=5)

    # Angle arc
    arc = np.linspace(0, theta_rad, 30)
    ax1.plot(0.3*np.cos(arc), 0.3*np.sin(arc), 'g-', linewidth=3)

    ax1.set_xlabel('Horizontal', fontsize=12)
    ax1.set_ylabel('Height (z)', fontsize=12)
    ax1.set_title(f'Dome Cross-Section\nθ = {elevation}°', fontsize=14)
    ax1.set_xlim(-1.2, 1.2)
    ax1.set_ylim(-0.1, 1.2)
    ax1.set_aspect('equal')
    ax1.annotate(f'z = sin({elevation}°) = {z:.2f}', xy=(0.1, z+0.05), fontsize=11,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    # === Plot 2: z vs elevation graph ===
    ax2 = axes[1]

    all_elevations = np.linspace(0, 90, 100)
    all_z = np.sin(np.radians(all_elevations))

    ax2.plot(all_elevations, all_z, 'b-', linewidth=2, label='z = sin(θ)')
    ax2.scatter([elevation], [z], c='red', s=200, zorder=5)
    ax2.axvline(x=elevation, color='red', linestyle='--', alpha=0.5)
    ax2.axhline(y=z, color='red', linestyle='--', alpha=0.5)

    ax2.set_xlabel('Elevation θ (degrees)', fontsize=12)
    ax2.set_ylabel('z = sin(θ)', fontsize=12)
    ax2.set_title('Height vs Elevation Angle', fontsize=14)
    ax2.set_xlim(0, 90)
    ax2.set_ylim(0, 1.1)
    ax2.grid(True, alpha=0.3)
    ax2.legend()

    # === Plot 3: CIE luminance ===
    ax3 = axes[2]

    all_cie = 1 + 2 * all_z

    ax3.fill_between(all_elevations, 1, all_cie, alpha=0.3, color='orange')
    ax3.plot(all_elevations, all_cie, 'orange', linewidth=2, label='CIE: L ∝ (1 + 2z)')
    ax3.scatter([elevation], [cie], c='red', s=200, zorder=5)
    ax3.axvline(x=elevation, color='red', linestyle='--', alpha=0.5)
    ax3.axhline(y=cie, color='red', linestyle='--', alpha=0.5)

    ax3.axhline(y=1, color='gray', linestyle=':', alpha=0.7)
    ax3.axhline(y=3, color='gray', linestyle=':', alpha=0.7)
    ax3.text(92, 1, 'Horizon', fontsize=9, va='center')
    ax3.text(92, 3, 'Zenith (3×)', fontsize=9, va='center')

    ax3.set_xlabel('Elevation θ (degrees)', fontsize=12)
    ax3.set_ylabel('CIE Luminance Factor', fontsize=12)
    ax3.set_title(f'Sky Brightness\nAt θ={elevation}°: {cie:.2f}× horizon', fontsize=14)
    ax3.set_xlim(0, 100)
    ax3.set_ylim(0.5, 3.5)
    ax3.grid(True, alpha=0.3)
    ax3.legend()

    plt.tight_layout()
    plt.show()

interact(
    plot_elevation_exploration,
    elevation=IntSlider(min=0, max=90, step=5, value=45, description='Elevation θ:')
);


interactive(children=(IntSlider(value=45, description='Elevation θ:', max=90, step=5), Output()), _dom_classes…

---

## 3. Understanding Azimuth (α) - "Which compass direction?"

The azimuth angle rotates around the vertical axis. Notice that azimuth **does NOT affect** the CIE luminance (which only depends on elevation/height).


In [ ]:
def plot_azimuth_exploration(azimuth=45, elevation=30):
    """Show how azimuth rotates the point around the vertical axis."""
    fig = plt.figure(figsize=(14, 6))

    theta_rad = np.radians(elevation)
    alpha_rad = np.radians(azimuth)

    x = np.cos(theta_rad) * np.cos(alpha_rad)
    y = np.cos(theta_rad) * np.sin(alpha_rad)
    z = np.sin(theta_rad)
    r = np.cos(theta_rad)

    # === 3D view ===
    ax1 = fig.add_subplot(121, projection='3d')

    u = np.linspace(0, 2*np.pi, 30)
    v = np.linspace(0, np.pi/2, 15)
    U, V = np.meshgrid(u, v)
    ax1.plot_wireframe(np.cos(V)*np.cos(U), np.cos(V)*np.sin(U), np.sin(V), alpha=0.15, color='gray')

    # Draw the elevation ring
    ring_alpha = np.linspace(0, 2*np.pi, 100)
    ax1.plot(r*np.cos(ring_alpha), r*np.sin(ring_alpha), [z]*100, 'g-', linewidth=2, alpha=0.5)
    ax1.scatter([x], [y], [z], c='red', s=200, zorder=5)
    ax1.plot([0, x], [0, y], [0, z], 'r-', linewidth=2)

    ax1.set_xlabel('X (East)')
    ax1.set_ylabel('Y (North)')
    ax1.set_zlabel('Z (Up)')
    ax1.set_title(f'3D View\nα={azimuth}° (azimuth), θ={elevation}° (elevation)')
    ax1.set_xlim(-1.2, 1.2)
    ax1.set_ylim(-1.2, 1.2)
    ax1.set_zlim(0, 1.2)

    # === Top-down view ===
    ax2 = fig.add_subplot(122)

    circle = np.linspace(0, 2*np.pi, 100)
    ax2.plot(np.cos(circle), np.sin(circle), 'k-', linewidth=2, alpha=0.3)
    ax2.plot(r*np.cos(circle), r*np.sin(circle), 'g--', linewidth=2, alpha=0.5)

    # Compass directions
    for angle, label in [(0, 'East'), (90, 'North'), (180, 'West'), (270, 'South')]:
        rad = np.radians(angle)
        ax2.plot([0, np.cos(rad)], [0, np.sin(rad)], 'gray', linestyle='--', alpha=0.5)
        ax2.text(1.15*np.cos(rad), 1.15*np.sin(rad), f'{label}\n({angle}°)', ha='center', fontsize=9)

    ax2.scatter([x], [y], c='red', s=200, zorder=5)
    ax2.plot([0, x], [0, y], 'r-', linewidth=2)

    # Azimuth arc
    arc_angles = np.linspace(0, alpha_rad, 50)
    ax2.plot(0.4*np.cos(arc_angles), 0.4*np.sin(arc_angles), 'b-', linewidth=3)
    ax2.annotate(f'α = {azimuth}°', xy=(0.5*np.cos(alpha_rad/2), 0.5*np.sin(alpha_rad/2)),
                fontsize=12, color='blue', fontweight='bold')

    ax2.set_xlabel('X', fontsize=12)
    ax2.set_ylabel('Y', fontsize=12)
    ax2.set_title(f'Top-Down View\nz = {z:.2f} (unchanged by azimuth!)', fontsize=14)
    ax2.set_aspect('equal')
    ax2.set_xlim(-1.4, 1.4)
    ax2.set_ylim(-1.4, 1.4)
    ax2.grid(True, alpha=0.2)

    plt.tight_layout()
    plt.show()

    print(f"🧭 Azimuth α = {azimuth}° rotates around the vertical axis")
    print(f"📐 The z-coordinate = {z:.2f} stays constant (only depends on elevation!)")

interact(
    plot_azimuth_exploration,
    azimuth=IntSlider(min=0, max=360, step=15, value=45, description='Azimuth α:'),
    elevation=IntSlider(min=0, max=90, step=15, value=30, description='Elevation θ:')
);


interactive(children=(IntSlider(value=45, description='Azimuth α:', max=360, step=15), IntSlider(value=30, des…